<a href="https://colab.research.google.com/github/gerazayisti/programmation-web/blob/main/doctranslator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.3 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import os
from google.cloud import translate

def diviser_pdf(fichier_pdf):
  """
  Divise un PDF en plusieurs fichiers selon le nombre de pages souhaité.

  Args:
      fichier_pdf (str): Le chemin d'accès au fichier PDF.
      nb_pages (int): Le nombre de pages par fichier.

  Returns:
      None
  """
  # Ouvre le fichier PDF et compte le nombre de pages
  with open(fichier_pdf, 'rb') as f:
      reader = PyPDF2.PdfReader(f)
      nb_total_pages = len(reader.pages)

  # Affiche le nombre de pages total et demande à l'utilisateur le nombre de pages par fichier
  print(f"Nombre total de pages: {nb_total_pages}")
  nb_pages = int(input("Entrez le nombre de pages par fichier: "))

  # Vérifie si le nombre de pages souhaité est valide
  if nb_pages <= 0 or nb_pages > nb_total_pages:
      raise ValueError("Le nombre de pages par fichier doit être positif et inférieur au nombre total de pages du PDF.")

  # Divise le PDF en plusieurs fichiers
  for i in range(0, nb_total_pages, nb_pages):
      nom_fichier = f"{fichier_pdf[:-4]}_{i + 1}_{i + nb_pages}.pdf"
      écrivain = PyPDF2.PdfWriter()

      for num_page in range(i, min(i + nb_pages, nb_total_pages)):
          page = reader.pages[num_page]
          écrivain.addPage(page)

      with open(nom_fichier, 'wb') as f:
          écrivain.write(f)

def traduire_pdf(fichier_pdf, langue_source, langue_cible):
  """
  Traduit un PDF en une autre langue en utilisant un modèle IA.

  Args:
      fichier_pdf (str): Le chemin d'accès au fichier PDF.
      langue_source (str): La langue source du texte.
      langue_cible (str): La langue cible du texte.

  Returns:
      None
  """
  # Initialise le client Google Cloud Translate
  client = translate.TranslationServiceClient()

  # Ouvre le fichier PDF et extrait le texte
  with open(fichier_pdf, 'rb') as f:
      reader = PyPDF2.PdfReader(f)
      texte = ""
      for page in reader.pages:
          texte += page.extract_text()

  # Propose trois options de traduction à l'utilisateur
  print("Langues de traduction disponibles:")
  print("1. Anglais")
  print("2. Français")
  print("3. Allemand")
  langue_cible = int(input("Entrez le numéro de la langue cible (1, 2 ou 3): "))

  # Détermine la langue cible selon le choix de l'utilisateur
  if langue_cible == 1:
      langue_cible = "en"
  elif langue_cible == 2:
      langue_cible = "fr"
  elif langue_cible == 3:
      langue_cible = "de"
  else:
      raise ValueError("Choix de langue invalide.")

  # Traduit le texte
  translation = client.translate(texte, source_language=langue_source, target_language=langue_cible)

  # Enregistre le texte traduit dans un nouveau fichier
  nom_fichier = f"{fichier_pdf[:-4]}_traduit_{langue_cible}.txt"
  with open(nom_fichier, 'w', encoding='utf-8') as f:
      f.write(translation.translated_text)

if __name__ == "__main__":
  # Demande à l'utilisateur le chemin d'accès au fichier PDF
  fichier_pdf = input("Entrez le chemin d'accès au fichier PDF: ")

  # Divise le PDF
  diviser_pdf(fichier_pdf)

  # Traduit les fichiers PDF divisés
  for nom_fichier in os.listdir():
      if nom_fichier.endswith(".pdf"):
          traduire_pdf(nom_fichier, langue_source="fr", langue_cible="en")  # Définir la langue source par défaut au français


Nombre total de pages: 2
